# 前端文件下载的几种方法

## 表单下载

通过提交表单形式可以达到下载后台返回的文件的效果, 同时适用于 `get` 和 `post` 方法

## a 标签下载 和 window.open 下载

这两类现在都需要给定的链接能够指向某个具体文件

## 通过 Blob 结合 a 标签 下载

这种适用于文件是后台根据数据动态生成的情况

[值得借鉴](https://blog.csdn.net/ljy123w/article/details/123438648)

## 实现

### 普通下载

需要注意的是, 这种下载需要响应头 `content-type: application/octet-stream` 或者是一种文件类型的 MIME

否则不会下载, 如果是 `application/json` `application/javascript` 这种则会直接在浏览器打开

同时这种 url 还应该返回 `content-length` 指明文件大小


In [ ]:
/**
 * 通过 a 标签实现下载文件
 * 需要一个指向目标文件的 url
 * @param {string} url 一个指向文件地址的 url
 */
function downloadWithATag(url) {
  let elink = document.createElement('a');
  elink.href = url;
  elink.click();
  elink = undefined;
}

/**
 * 这种方式和 上面下载原理相同
 * 只是会新开一个窗口, 并且在下载后关闭这个窗口
 * @param url
 */
function downloadWithWindowOpen(url) {
  window.open(url, '__blank');
}


### 通过接口动态下载


In [ ]:
import URI from 'urijs';

function downloadWithXHR(url, method = 'GET', params) {
  url = new URI(url);
  let xhr = new XMLHttpRequest();
  xhr.open(method, url);
  xhr.responseType = 'blob';
  xhr.onload = function () {
    if (this.status === 200 || this.status === 304) {
      let fileReader = new FileReader();

      fileReader.onload = function () {
        /* 返回的是json数据, 说明报错 */
        if (
          xhr.getResponseHeader('content-type').includes('application/json')
        ) {
          let data = JSON.parse(this.result);
          alert(data.message || '');
        } else {
          let disposition = xhr.getResponseHeader('content-disposition');
          let filename = disposition.split(';')[1].split('=')[1];
          filename = decodeURIComponent(filename);

          let elink = document.createElement('a');
          elink.download = filename;
          elink.style.display = 'none';
          elink.href = URL.createObjectURL(xhr.response);
          elink.click();
          URL.revokeObjectURL(elink.href); // 释放URL 对象
          elink = undefined;
        }
      };
      fileReader.readAsText(this.response);
    }else {
      alert('接口失败')
    }
  };
  switch (method) {
    case 'GET':
      // 使用 URI 库处理 请求参数
      url.addSearch(params);
      xhr.send();
      break;

    case 'POST':
      // 这里具体取决于发送要发什么什么类型数据
      xhr.send(JSON.stringify(params));
      break;
  }
}
